In [1]:
%run ../../v4/misc/data_access.py

get_project_folder(): /home/marcel/projects/eco/
get_data_folder(): /mnt/drive1/data/eco/
get_model_folder(): /mnt/drive1/data/eco/models/


In [7]:
# final-json to feather
import json
import codecs
import os
import pandas as pd

def read_json_file(file_name):
   with codecs.open(file_name,encoding='utf-8') as fin:
       return json.loads(fin.read())

json = read_json_file(get_project_folder() + 'src/python/notebooks/log-final.json')
base_path = json['folder_path']

authors = {}

for key in json['file_descriptors']:
    values = json['file_descriptors'][key]
    rel_path = values['rel_path']
    file_name = values['file_name']
    author_name = values['author_name']
    abs_path = os.path.join(base_path + rel_path, file_name)
    if author_name not in authors:
        authors[author_name] = []
        
    authors[author_name].append(abs_path)
        
# dont start this unless you have some time :)
dataframes = []
# saving dataframes
for key in authors:
    if len(authors[key]) > 20:
        print(key + ' ' + str(len(authors[key])))
        for path in authors[key]:
            print(path)
            df = v3_txt_to_df(path, key)
            dataframes.append(df)
            print(len(dataframes))

print(len(dataframes))
bigdata = pd.concat(dataframes, ignore_index=True)
bigdata.to_feather(get_model_folder() +  'parsed_v3_log-final_bigger_than_20_incl_filename.feather')

Garland,David 24
/mnt/drive1/data/eco/NAIL_DATAFIELD_txt/parsed_v3/law_legal_theory_prison_ip/Garland - Review - The Commonplace and the Catastrophic - Interpretations of Crime in Late M_valid.txt
[('publications', 0.5955274105072021), ('essays', 0.5703297853469849), ('journal', 0.5485131144523621), ('monographs', 0.5223984122276306), ('rethinking', 0.522263765335083), ('essay', 0.5210254788398743), ('review', 0.5072212815284729), ('ecocriticism', 0.5068739056587219), ('sociological', 0.5052769184112549), ('theory', 0.4989697337150574)]
[(',', 0.6477832794189453), ('rethinking', 0.5640451908111572), ('modernity', 0.5625160932540894), ('critique', 0.5615260601043701), (':', 0.5593869686126709), ('.', 0.553613007068634), ('morality', 0.5529439449310303), ('aesthetics', 0.5367676615715027), ('theory', 0.5334801077842712), ('deviance', 0.532272458076477)]
[(':', 0.6701797842979431), ('&', 0.6192981600761414), ('schuster', 0.5365140438079834), ('.', 0.5300980806350708), ('162', 0.5242410302

[('seem', 0.6117356419563293), ('choice', 0.5608702898025513), ('thinking', 0.5228046178817749), ('think', 0.5170756578445435), ('choices', 0.5151342153549194), ('sense', 0.5048015117645264), ('behavior', 0.49907398223876953), ('necessarily', 0.4947967231273651), ('thought', 0.48484599590301514), ('conversely', 0.4846529960632324)]
[('pathos', 0.5680319666862488), ('sense', 0.5642975568771362), ('introspection', 0.5421034693717957), ('nuanced', 0.5330686569213867), ('existential', 0.5324230194091797), ('expression', 0.5309065580368042), ('interpretation', 0.5222952961921692), ('aesthetic', 0.5178512334823608), ('idealization', 0.515680730342865), ('inwardness', 0.5144637823104858)]
[('.', 0.6662648916244507), (',', 0.663355827331543), ('nature', 0.6581265926361084), ('stress', 0.6457287073135376), ('deviance', 0.6424077749252319), ('behavior', 0.6393845677375793), ('expression', 0.6183334589004517), ('generalized', 0.6181088089942932), ('perception', 0.5990921258926392), ('psychologica

[('something', 0.6728827953338623), ('everything', 0.6623753905296326), ('anything', 0.6588799953460693), ('everyone', 0.6367336511611938), ('nothing', 0.6267225742340088), ('someone', 0.5706896781921387), ('things', 0.5703892111778259), ('truth', 0.5672838091850281), ('message', 0.5618033409118652), ('what', 0.5609490871429443)]
[(',', 0.6712071895599365), ('.', 0.6438119411468506), ('nature', 0.6417810320854187), ('sense', 0.6203420162200928), ('inherently', 0.6002553701400757), ('moral', 0.5998080968856812), ('perception', 0.591296374797821), ('morality', 0.5775536298751831), ('often', 0.575863778591156), ('inherent', 0.57243812084198)]
[('.', 0.6497284770011902), ('way', 0.6403625011444092), (',', 0.5580343008041382), ('and', 0.524136483669281), ('sense', 0.5168054103851318), ('or', 0.5014296174049377), ('ability', 0.4973866045475006), ('ways', 0.4939349293708801), ('emotions', 0.49175575375556946), ('situations', 0.4874339997768402)]
[(',', 0.7136486768722534), ('.', 0.66468369960

KeyboardInterrupt: 

In [ ]:
# 1

import gensim
from nltk.corpus import stopwords
import numpy as np


model = gensim.models.Word2Vec.load(get_model_folder() + 'word2vec_models/wiki_plus_v3_valid_combined.txt_numpy.w2vmodel')


In [6]:
#1.1

# sentence to vector via w2v
def to_vector(sentence):
    sentence = sentence.lower()
    # not using english stop words, only using content words
    words = [word for word in sentence.split() if (word not in stopwords.words("english") and word in model.vocab)]

    vectors = []
    for word in words:
        vectors.append(model[word])

    vector = np.mean(vectors, axis=0)
    print(model.similar_by_vector(vector))
    return vector


# converts a v4 text file into feather file with w2v vec saved
def v4_txt_to_feather(path, feather_name):
    data = open(path).readlines()

    row_list = []
    for line in data:
        sentence = line.split(';')[0].rstrip()
        vec = to_vector(sentence)
        new_row = {}
        index = 0
        try:
            for i in vec:
                new_row['p'+str(index)] = i
                index += 1

            new_row['sentence'] = sentence
            row_list.append(new_row)
        except:
            # only happening sometimes. \o/
            pass

    df = pd.DataFrame(row_list)
    df.to_feather(feather_name)


def v3_txt_to_df(path, author):
    data = open(path).readlines()
    path, filename = os.path.split(path)
    row_list = []
    for line in data:
        vec = to_vector(line)
        new_row = {}
        index = 0
        try:
            for i in vec:
                new_row['p'+str(index)] = i
                index += 1

            new_row['sentence'] = line
            new_row['author'] = author
            new_row['filename'] = filename
            row_list.append(new_row)
        except:
            # only happening sometimes. \o/
            pass

    return pd.DataFrame(row_list)

    
#v4_txt_to_feather('/mnt/drive1/data/eco/NAIL_DATAFIELD_txt/parsed_v4/own_collection_valid.txt', 'out.feather')

In [3]:
# 2

import feather

df = feather.read_dataframe(get_model_folder() + 'parsed_v3_log-final_bigger_than_20.feather')
_df = df

In [4]:
# 3
# get the closest sentences to this one
import pandas as pd
# from tqdm import trange
from IPython.core.display import display, HTML

pd.set_option('max_colwidth',200)

def get_similar(df, sentence):

    sentence_vec = to_vector(sentence)

    dist = 0.065
    ldf = df
    for i in range(300):
        tdf = ldf[(ldf['p'+str(i)] > sentence_vec[i]-dist) & (ldf['p'+str(i)] < sentence_vec[i]+dist)]
#         print(len(tdf.index))
        num_sentences = len(tdf.index)
        if num_sentences > 0:
            ldf = tdf
        if num_sentences < 5:
            break

    display(ldf[['sentence','author']])
    
# get_similar(df,'power war fight pain kill')
get_similar(df, 'politic england germany kitchen asia')


[('europe', 0.6680523157119751), ('germany', 0.6534768342971802), ('asia', 0.5866613388061523), ('france', 0.5456572771072388), ('africa', 0.5350488424301147), ('england', 0.5349924564361572), ('japan', 0.5180532336235046), ('india', 0.5083079934120178), ('scandinavia', 0.5061678886413574), ('russia', 0.5046881437301636)]


,sentence,author
44080,It was with great honour that a small team embraced the gigantic role of coordinating the visit of Professor Chomsky to Australia in January 1995 .\n,"Chomsky,Noam"
52424,"It would also carry infection to Africa through Asia Minor and Egypt , and to Europe through Italy and France , already threatened by the strongest domestic Communist parties in Western Europe .\n","Chomsky,Noam"
190248,French supremacy had been established with the French occupation of Fez in 1911 and in November of that year France and Germany signed the entente cordiale establishing their spheres of influence ...,No Author
